In [1]:
# import common libraries

import sys
import time
import json

In [2]:
# Import the pyspark libraries

from pyspark import SparkContext
from pyspark.streaming import  StreamingContext
from pyspark.sql import  SQLContext

In [3]:
# Kafka Server configuration

# Import Kafka libraries
from kafka import KafkaConsumer, KafkaProducer

kafka_hostname = '35.239.57.13'
kafka_port = '9092'
kafka_bootstrap_server = kafka_hostname + ':' + kafka_port

In [4]:
# Register to Kafka as a Producer

def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=[kafka_bootstrap_server], api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka')
        print(str(ex))
    finally:
        return _producer

In [5]:
# Publishes the message to Kafka Topic

def publishToKafka(topic_name, data):
    producer = connect_kafka_producer()
    text_to_send = bytes(data, encoding='utf-8')
    try:
        producer.send(topic_name,value=text_to_send)
        producer.flush()
        print(f"Message delivered to Kafka Topic - {topic_name}")
    except Exception as ex:
        #print("Message sending failed!!")
        print(ex)
    producer.close()

In [6]:
# Use Case 1

from nltk import word_tokenize
from textblob import TextBlob
import re

def sentimentCategoryBlob(score):
    if score == 0.0:
        category = "neutral"
    elif score > 0.0:
        category = "positive"
    elif score < 0.0:
        category = "negative"
        
    return category

# Cleans the tweet Text
def clean_tweet(text):
    """
    Removes the junk characters and tweet names
    """
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ",text).split())

# Applies Blob Algorithm to find sentiments to each texts
def blobSentimentAnalysis(text):
    
    text = clean_tweet(text)
    analysis = TextBlob(text)
    
    return analysis.sentiment.polarity

def findBlobSentiment(text):
    sentiment = blobSentimentAnalysis(str(text))
    #print(sentiment)
    category = sentimentCategoryBlob(sentiment)
    return category

def tweetsPerSentiment(dic) :
    topic_name = "blob_sentiments"
    senti_map = {"positive":0, "negative":0, "neutral": 0}
    for key in dic.keys():
        senti_map[key] = dic[key]
    #print(dic)
    print(senti_map)
    if dic:
        publishToKafka(topic_name, json.dumps(senti_map))
        #pass


In [7]:
# Use Case 2

import re

def electionMap(tweet):
    parties_list = list()
    if tweet: 
        tweet = tweet.lower()
        if re.search(r"shivsena|janata|jdu|bjp|modi|namo|chowkidar|shah|nda", tweet):
            parties_list.insert(0,"BJP")
        if re.search(r"indiancongress|raga|gandhi|inc|sonia|congress|rahul|priyanka|gatbandhan|nyay", tweet):
            parties_list.insert(0,"INC")
        if re.search(r"aap|kejri|arvind", tweet):
            parties_list.insert(0,"AAP") 
        if re.search(r"mamata|cpi|kanhaiya|bsp|samajwadi|tmc|trinamool|dmk|mns|bjd|samajwadi|yadav|gatbandhan|naidu|kalyan", tweet):
            parties_list.insert(0,"Others")
    return parties_list

def tweetsPerParty(dic):
    topic_name = "election_parties"
    election_map = {"BJP": 0, "INC": 0, "AAP": 0, "Others": 0}
    for key in dic.keys():
        election_map[key] = dic[key]
    print(election_map)
    if dic:
        publishToKafka(topic_name, json.dumps(election_map))
        #pass

In [8]:
# Use Case 3

# Find top 6 hashtags
def topTrendingHashTags(records):
    topic_name = "trending_hashtags"
    top_k_tweets = {}
    if records:
        top_k_tweets = dict(records)
    print(top_k_tweets)
    if records:
        publishToKafka(topic_name, json.dumps(top_k_tweets))

In [9]:
# Convert a tweet stream to JSON
def convert_tweet(tweet):
    if not tweet:
        tweet = "{}"
    return json.loads(tweet)

# Extract Tweet Text from tweet JSON
def extractTweetText(tweet):
    try:
        text = tweet['text']
        
    except KeyError as ex:
        print("No key attribute - text in the JSON")
        text = ""
    return text

# Extract Hashtags from a tweet JSON
def extractTweetHashtags(tweet):
    try:
        # hashtags is a list
        hashtags = tweet['entities']['hashtags']
        
    except KeyError as ex:
        print("No key attribute - entities:hashtags in the JSON")
        hashtags = ""
        
    return hashtags

In [10]:
if __name__ == '__main__':
    if len(sys.argv) != 3:
        print(f"Usage: python <file_name> <netcat_host_name> <netcat_port>")
        exit(-1)

    nc_host_name = "localhost"
    nc_port = 9999
    
    # batch interval in seconds
    batch_interval = 1
    window_length = 15 * batch_interval
    sliding_window = 1 * batch_interval

    spc = SparkContext.getOrCreate()
    spc.setLogLevel("ERROR")
    stc = StreamingContext(spc, batch_interval)
    stc.checkpoint("/tmp/checkpoint")

    tweetsDStream = stc.socketTextStream(nc_host_name, nc_port)
    
    try:
        
        # Converts tweet stream to json object
        tstream = tweetsDStream.map(lambda tweet: convert_tweet(tweet))
        #tstream = tstream.filter(lambda tweet: filterOnLang(tweet))
 
        # Extracts the text from tweet json
        textstream = tstream.map(lambda tweet: extractTweetText(tweet))
        
        # Counts the number of records in the stream batch
        tstream.count().map(lambda x:'Tweets in this batch: %s' % x).pprint()
        
        # Counts the number of records in the streaming window
        tstream.countByWindow(window_length,sliding_window).map(lambda x: "Tweets in this 1 minute window: %s" %x).pprint()
        
        ################## Use Case 2: Tweets per party ####################
        
        # Returns a map of election parties and their no. of tweets
        election = textstream.flatMap(lambda tweet: electionMap(tweet))
        election.countByValueAndWindow(window_length,sliding_window).foreachRDD(lambda rdd: tweetsPerParty(dict(rdd.collect())))
      
        ################## Use Case 3: Trending Hashtags ####################
        
        # Find the dictionary of top 6 Trending Hashtags in each tweets
        hashtagstream = tstream.map(lambda tweet: extractTweetHashtags(tweet))
        hashtagstream = hashtagstream.flatMap(lambda hashtags: [ hashtag['text'] for hashtag in hashtags if hashtag ])
        hashtagstream.countByValueAndWindow(window_length,sliding_window).transform(lambda rdd: rdd.sortBy(lambda x: -x[1])).foreachRDD(lambda rdd: topTrendingHashTags(rdd.take(6)))
    
        ################## Use Case 1: Sentiment Analysis ####################
        
        # Finds the Sentiments of the tweets using Blob based algorithm
        sentimentstream = textstream.map(lambda text: findBlobSentiment(text))
        sentimentstream.countByValueAndWindow(window_length,sliding_window).foreachRDD(lambda rdd: tweetsPerSentiment(dict(rdd.collect())))
        
        stc.start()
        stc.awaitTermination()
        
    except Exception as ex:
        print(ex)

-------------------------------------------
Time: 2019-04-19 16:17:29
-------------------------------------------

-------------------------------------------
Time: 2019-04-19 16:17:29
-------------------------------------------

{'BJP': 0, 'INC': 0, 'AAP': 0, 'Others': 0}
{}
{'positive': 0, 'negative': 0, 'neutral': 0}
-------------------------------------------
Time: 2019-04-19 16:17:30
-------------------------------------------
Tweets in this batch: 138

-------------------------------------------
Time: 2019-04-19 16:17:30
-------------------------------------------
Tweets in this 1 minute window: 138

{'BJP': 58, 'INC': 15, 'AAP': 5, 'Others': 13}
Message delivered to Kafka Topic - election_parties
{'HemantKarkare': 4, 'MuellerReport': 3, 'LokSabhaElections2019': 1, 'FerdousAhmed': 1, 'nontfam': 1, 'Nontfamphoto': 1}
Message delivered to Kafka Topic - trending_hashtags
{'positive': 40, 'negative': 24, 'neutral': 74}
Message delivered to Kafka Topic - blob_sentiments
--------------

{'positive': 164, 'negative': 85, 'neutral': 291}
Message delivered to Kafka Topic - blob_sentiments
-------------------------------------------
Time: 2019-04-19 16:17:42
-------------------------------------------
Tweets in this batch: 28

-------------------------------------------
Time: 2019-04-19 16:17:42
-------------------------------------------
Tweets in this 1 minute window: 568

{'BJP': 201, 'INC': 62, 'AAP': 17, 'Others': 48}
Message delivered to Kafka Topic - election_parties
{'HemantKarkare': 7, 'MuellerReport': 5, 'OppnBreakUp': 5, 'LokSabhaElections2019': 4, 'BJP': 3, 'SadhviPragya': 3}
Message delivered to Kafka Topic - trending_hashtags
{'positive': 173, 'negative': 88, 'neutral': 307}
Message delivered to Kafka Topic - blob_sentiments
-------------------------------------------
Time: 2019-04-19 16:17:43
-------------------------------------------
Tweets in this batch: 34

-------------------------------------------
Time: 2019-04-19 16:17:43
---------------------------

{'BJP': 191, 'INC': 58, 'AAP': 21, 'Others': 39}
Message delivered to Kafka Topic - election_parties
{'HemantKarkare': 13, 'SadhviPragya': 7, 'MuellerReport': 5, 'BJP': 3, 'OppnBreakUp': 3, 'TheBigFight': 2}
Message delivered to Kafka Topic - trending_hashtags
{'positive': 162, 'negative': 86, 'neutral': 320}
Message delivered to Kafka Topic - blob_sentiments
-------------------------------------------
Time: 2019-04-19 16:17:55
-------------------------------------------
Tweets in this batch: 52

-------------------------------------------
Time: 2019-04-19 16:17:55
-------------------------------------------
Tweets in this 1 minute window: 593

{'BJP': 199, 'INC': 58, 'AAP': 20, 'Others': 44}
Message delivered to Kafka Topic - election_parties
{'HemantKarkare': 14, 'SadhviPragya': 7, 'MuellerReport': 5, 'BJP': 3, 'TheBigFight': 2, 'PragyaThakur': 2}
Message delivered to Kafka Topic - trending_hashtags
{'positive': 166, 'negative': 94, 'neutral': 333}
Message delivered to Kafka Topic - 

Message delivered to Kafka Topic - trending_hashtags
{'positive': 198, 'negative': 120, 'neutral': 418}
Message delivered to Kafka Topic - blob_sentiments
-------------------------------------------
Time: 2019-04-19 16:18:07
-------------------------------------------
Tweets in this batch: 32

-------------------------------------------
Time: 2019-04-19 16:18:07
-------------------------------------------
Tweets in this 1 minute window: 731

{'BJP': 230, 'INC': 67, 'AAP': 28, 'Others': 58}
Message delivered to Kafka Topic - election_parties
{'HemantKarkare': 18, 'SadhviPragya': 8, 'LokSabhaElections2019': 4, 'MuellerReport': 4, 'PragyaThakur': 2, 'CompleteExoneration': 2}
Message delivered to Kafka Topic - trending_hashtags
{'positive': 205, 'negative': 120, 'neutral': 406}
Message delivered to Kafka Topic - blob_sentiments
-------------------------------------------
Time: 2019-04-19 16:18:08
-------------------------------------------
Tweets in this batch: 49

------------------------

-------------------------------------------
Time: 2019-04-19 16:18:19
-------------------------------------------
Tweets in this 1 minute window: 688

{'BJP': 229, 'INC': 72, 'AAP': 24, 'Others': 35}
Message delivered to Kafka Topic - election_parties
{'HemantKarkare': 22, 'SadhviPragya': 10, 'BJP': 6, 'PragyaThakur': 5, 'LokSabhaElections2019': 4, 'MuellerReport': 4}
Message delivered to Kafka Topic - trending_hashtags
{'positive': 190, 'negative': 115, 'neutral': 383}
Message delivered to Kafka Topic - blob_sentiments
-------------------------------------------
Time: 2019-04-19 16:18:20
-------------------------------------------
Tweets in this batch: 59

-------------------------------------------
Time: 2019-04-19 16:18:20
-------------------------------------------
Tweets in this 1 minute window: 706

{'BJP': 239, 'INC': 77, 'AAP': 23, 'Others': 35}
Message delivered to Kafka Topic - election_parties
{'HemantKarkare': 20, 'SadhviPragya': 7, 'PragyaThakur': 6, 'BJP': 6, 'LokSabhaEle

...

-------------------------------------------
Time: 2019-04-19 16:23:20
-------------------------------------------
Tweets in this 1 minute window: 635

{'BJP': 200, 'INC': 63, 'AAP': 29, 'Others': 43}
Message delivered to Kafka Topic - election_parties
{'HemantKarkare': 10, 'SadhviPragya': 7, 'pragyaThakur': 3, 'BJP': 3, 'BSP': 2, 'PragyaSinghThakur': 2}
Message delivered to Kafka Topic - trending_hashtags
{'positive': 142, 'negative': 118, 'neutral': 375}
Message delivered to Kafka Topic - blob_sentiments
-------------------------------------------
Time: 2019-04-19 16:23:21
-------------------------------------------
Tweets in this batch: 58

-------------------------------------------
Time: 2019-04-19 16:23:21
-------------------------------------------
Tweets in this 1 minute window: 661

{'BJP': 205, 'INC': 63, 'AAP': 32, 'Others': 47}
Message delivered to Kafka Topic - election_parties
{'HemantKarkare': 8, 'SadhviPragya': 6, 'AbkiBaarKiskiSarkar': 3, 'BSP': 2, 'PragyaSinghThakur': 

{'HemantKarkare': 10, 'MuellerReport': 6, 'SadhviPragya': 6, 'AbkiBaarKiskiSarkar': 3, 'BJP': 3, 'WATCH': 3}
Message delivered to Kafka Topic - trending_hashtags
{'positive': 171, 'negative': 121, 'neutral': 354}
Message delivered to Kafka Topic - blob_sentiments
-------------------------------------------
Time: 2019-04-19 16:23:33
-------------------------------------------
Tweets in this batch: 50

-------------------------------------------
Time: 2019-04-19 16:23:33
-------------------------------------------
Tweets in this 1 minute window: 650

{'BJP': 208, 'INC': 64, 'AAP': 22, 'Others': 52}
Message delivered to Kafka Topic - election_parties
{'HemantKarkare': 12, 'MuellerReport': 6, 'SadhviPragya': 5, 'BJP': 4, 'AbkiBaarKiskiSarkar': 3, 'WATCH': 3}
Message delivered to Kafka Topic - trending_hashtags
{'positive': 180, 'negative': 122, 'neutral': 348}
Message delivered to Kafka Topic - blob_sentiments
-------------------------------------------
Time: 2019-04-19 16:23:34
----------

-------------------------------------------
Time: 2019-04-19 16:23:45
-------------------------------------------
Tweets in this 1 minute window: 669

{'BJP': 224, 'INC': 61, 'AAP': 28, 'Others': 46}
Message delivered to Kafka Topic - election_parties
{'HemantKarkare': 15, 'BJP': 8, 'AbkiBaarKiskiSarkar': 7, 'SadhviPragya': 7, 'pragyaThakur': 6, 'PragyaThakur': 4}
Message delivered to Kafka Topic - trending_hashtags
{'positive': 180, 'negative': 116, 'neutral': 373}
Message delivered to Kafka Topic - blob_sentiments
-------------------------------------------
Time: 2019-04-19 16:23:46
-------------------------------------------
Tweets in this batch: 46

-------------------------------------------
Time: 2019-04-19 16:23:46
-------------------------------------------
Tweets in this 1 minute window: 680

{'BJP': 227, 'INC': 67, 'AAP': 29, 'Others': 47}
Message delivered to Kafka Topic - election_parties
{'HemantKarkare': 15, 'SadhviPragya': 8, 'AbkiBaarKiskiSarkar': 7, 'pragyaThakur': 7, 

{'HemantKarkare': 7, 'SadhviPragya': 7, 'LokSabhaElections2019': 5, 'pragyaThakur': 4, 'BJP': 4, 'MuellerReport': 3}
Message delivered to Kafka Topic - trending_hashtags
{'positive': 175, 'negative': 137, 'neutral': 349}
Message delivered to Kafka Topic - blob_sentiments
-------------------------------------------
Time: 2019-04-19 16:23:58
-------------------------------------------
Tweets in this batch: 31

-------------------------------------------
Time: 2019-04-19 16:23:58
-------------------------------------------
Tweets in this 1 minute window: 641

{'BJP': 194, 'INC': 70, 'AAP': 22, 'Others': 55}
Message delivered to Kafka Topic - election_parties
{'SadhviPragya': 7, 'HemantKarkare': 6, 'LokSabhaElections2019': 5, 'BJP': 4, 'pragyaThakur': 3, 'MuellerReport': 3}
Message delivered to Kafka Topic - trending_hashtags
{'positive': 170, 'negative': 137, 'neutral': 334}
Message delivered to Kafka Topic - blob_sentiments
-------------------------------------------
Time: 2019-04-19 16:

-------------------------------------------
Time: 2019-04-19 16:24:10
-------------------------------------------
Tweets in this 1 minute window: 609

{'BJP': 163, 'INC': 57, 'AAP': 20, 'Others': 37}
Message delivered to Kafka Topic - election_parties
{'HemantKarkare': 10, 'SadhviPragya': 9, 'pragyaThakur': 4, 'MuellerReport': 4, 'Trump': 3, 'BJP': 3}
Message delivered to Kafka Topic - trending_hashtags
{'positive': 167, 'negative': 116, 'neutral': 326}
Message delivered to Kafka Topic - blob_sentiments
-------------------------------------------
Time: 2019-04-19 16:24:11
-------------------------------------------
Tweets in this batch: 45

-------------------------------------------
Time: 2019-04-19 16:24:11
-------------------------------------------
Tweets in this 1 minute window: 620

{'BJP': 165, 'INC': 56, 'AAP': 18, 'Others': 38}
Message delivered to Kafka Topic - election_parties
{'HemantKarkare': 10, 'SadhviPragya': 10, 'pragyaThakur': 4, 'MuellerReport': 4, 'Trump': 3, 'BJP':